In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, kendalltau, spearmanr
from matplotlib import style
style.use('seaborn')
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from statistics import mean
from ast import literal_eval

In [ ]:
Netflix=pd.read_csv("NETFLIX.csv",sep=";")
Netflix.tail(80)
  



,Título_id,Título,Episodios,Rating,Votos,Resumen,Fecha Estreno,Tags,Idioma,Género,cast,país,Productora,Director/Creador
5215,6047,Die Goldene Kamera 2017,No episodes,8.7,15,The Golden Camera 2017.,4 March 2017,"['awards show', 'tv special']",['German'],['News'],"['Steven Gätjen', 'James Blunt', 'Leonard Caro...",['Germany'],"[' Riverside Entertainment', ' Funke Mediengr...",['Ladislaus Kiraly']
5216,6050,Cedonia Avenue,No episodes,4.2,11,Follows the ups and downs of the life of a you...,27 April 2015,['game'],['English'],['Drama'],"['Dwayne Hurt', 'Durrell Overby', 'Jaleel Hurt']",['Usa'],[' Amazon Instant Video'],['Dwayne Hurt']
5217,6060,Autistic Driving School,No episodes,5.3,10,A fascinating portrait of upcoming young drive...,29 April 2010,[' Plot Keywords submission guide.\n'],['English'],['Documentary'],['Missing'],['Uk'],[' Firecracker Films'],['Osca Humphreys']
5218,6065,Crap Shoot: The Documentary,No episodes,3.8,183,One of America\'s top research scientists and ...,5 May 2007,"['hollywood', 'science', 'interview', 'actress...",['English'],['Documentary'],"['Kenneth R. Close', 'James E. Horton', 'Joey ...",['Usa'],"[' K.C. Films', ' Low Fidelity Productions (I)']",['Kenneth R. Close']
5219,6070,High Strange New Mexico,No episodes,6.6,7,Starting from the premise that New Mexico hold...,29 April 1997,[' Plot Keywords submission guide.\n'],['English'],['Documentary'],['Missing'],['Usa'],['Missing'],['James Lujan']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5290,10022,How I Met Your Mother,208 episodes,8.3,612244,"Ted Mosby sits down with his kids, to tell the...",19 September 2005,"['group of friends', 'friendship', 'best frien...","['English', 'Persian', 'Chinese']","['Comedy', 'Romance']","['Josh Radnor', 'Jason Segel', 'Cobie Smulders...",['Usa'],"[' 20Th Century Fox Television', ' Bays Thoma...",['Carter Bays']
5291,10023,Sexify,8 episodes,6.4,728,To build an innovative sex app and win a tech ...,28 April 2021,"['partial female nudity', 'male explicit nudit...",['Polish'],"['Comedy', 'Drama']","['Aleksandra Skraba', 'Maria Sobocinska', 'San...",['Poland'],[' Akson Studio'],['Aleksandra Skraba']
5292,10024,Ozark,30 episodes,8.4,196614,"The Byrdes and their teenage kids, Charlotte a...",21 July 2017,"['netflix original', 'missouri', 'mexico', 'dr...",['English'],"['Crime', 'Drama', 'Thriller']","['Jason Bateman', 'Laura Linney', 'Sofia Hubli...",['Usa'],"[' Zero Gravity Management', ' Media Rights C...",['Bill Dubuque']
5293,10026,365 Days,No episodes,3.2,57342,"Massimo Torricelli, a young and handsome boss ...",7 June 2020,"['oral sex', 'sex scene', 'in love with kidnap...","['Polish', 'English', 'Italian']","['Drama', 'Romance']","['Anna Maria Sieklucka', 'Michele Morrone', 'B...",['Poland'],"[' Ekipa', ' Future Space', ' Next Film']",['Barbara Bialowas']


In [ ]:
Netflix = Netflix.rename(columns={'Rating': 'Calificación', 'cast': 'Actores','Tags':'Etiquetas', 'país':"País"})
Netflix

,Título_id,Título,Episodios,Calificación,Votos,Resumen,Fecha Estreno,Etiquetas,Idioma,Género,Actores,País,Productora,Director/Creador
0,0,The Vampire Diaries,171 episodes,7.7,287747,"After centuries of quarreling, Stefan and Damo...",January 2010,"['vampire', 'high school', 'teen romance', 'ba...",['English'],"['Drama', 'Fantasy', 'Horror', 'Mystery', 'Rom...","['Paul Wesley', 'Ian Somerhalder', 'Kat Graham...",['Usa'],"[' Outerbanks Entertainment', ' Alloy Enterta...",['Julie Plec']
1,1,Yasuke,6 episodes,6.3,1157,He came from Africa and fought alongside a mig...,29 April 2021,"['anime', 'samurai', 'legend', 'sword', 'black...","['Japanese', 'English']","['Animation', 'Action', 'Adventure', 'Drama', ...","['Lakeith Stanfield', 'Maya Tanida', 'William ...","['Japan', 'Usa']","[' Mappa', ' Netflix']",['Lesean Thomas']
2,2,The Last Kingdom,40 episodes,8.4,89263,"The year is 872, and many of the separate king...",28 December 2015,"['kingdom', 'danish', 'anglo saxon', 'male rea...","['English', 'Latin']","['Action', 'Drama', 'History']","['Alexander Dreymon', 'Eliza Butterworth', 'Ia...",['Uk'],[' Carnival Film & Television'],['Alexander Dreymon']
3,3,The Queen's Gambit,7 episodes,8.6,304534,"Nine year-old orphan Beth Harmon is quiet, sul...",23 October 2020,"['addiction', 'chess', 'female protagonist', '...","['French', 'Spanish', 'Russian', 'English']",['Drama'],"['Anya Taylor-Joy', 'Chloe Pirrie', 'Bill Camp...",['Usa'],"[' Flitcraft', ' Wonderful Films', ' Netflix']",['Scott Frank']
4,4,Sons of Anarchy,92 episodes,8.5,264918,"Sons of Anarchy, aka SAMCRO, is a motorcycle c...",15 February 2010,"['motorcycle', 'gun running', 'street gang', '...",['English'],"['Crime', 'Drama', 'Thriller']","['Charlie Hunnam', 'Katey Sagal', 'Mark Boone ...",['Usa'],"[' Sutterink', ' Linson Entertainment', ' Fo...",['Kurt Sutter']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5290,10022,How I Met Your Mother,208 episodes,8.3,612244,"Ted Mosby sits down with his kids, to tell the...",19 September 2005,"['group of friends', 'friendship', 'best frien...","['English', 'Persian', 'Chinese']","['Comedy', 'Romance']","['Josh Radnor', 'Jason Segel', 'Cobie Smulders...",['Usa'],"[' 20Th Century Fox Television', ' Bays Thoma...",['Carter Bays']
5291,10023,Sexify,8 episodes,6.4,728,To build an innovative sex app and win a tech ...,28 April 2021,"['partial female nudity', 'male explicit nudit...",['Polish'],"['Comedy', 'Drama']","['Aleksandra Skraba', 'Maria Sobocinska', 'San...",['Poland'],[' Akson Studio'],['Aleksandra Skraba']
5292,10024,Ozark,30 episodes,8.4,196614,"The Byrdes and their teenage kids, Charlotte a...",21 July 2017,"['netflix original', 'missouri', 'mexico', 'dr...",['English'],"['Crime', 'Drama', 'Thriller']","['Jason Bateman', 'Laura Linney', 'Sofia Hubli...",['Usa'],"[' Zero Gravity Management', ' Media Rights C...",['Bill Dubuque']
5293,10026,365 Days,No episodes,3.2,57342,"Massimo Torricelli, a young and handsome boss ...",7 June 2020,"['oral sex', 'sex scene', 'in love with kidnap...","['Polish', 'English', 'Italian']","['Drama', 'Romance']","['Anna Maria Sieklucka', 'Michele Morrone', 'B...",['Poland'],"[' Ekipa', ' Future Space', ' Next Film']",['Barbara Bialowas']


In [ ]:
Netflix1=Netflix[['Título', 'Calificación','Votos','Actores','Productora',"Director/Creador","Etiquetas","Género"]]

In [ ]:

columnas= ['Actores','Productora',"Director/Creador","Etiquetas","Género"]
for i in columnas:
  Netflix1[i]=Netflix1[i].apply(literal_eval)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
for i in range(len(Netflix1['Etiquetas'])):
  x=Netflix1['Etiquetas'][i][0].rstrip()
  Netflix1['Etiquetas'][i][0]=x

Netflix1.tail(80)

,Título,Calificación,Votos,Actores,Productora,Director/Creador,Etiquetas,Género
5215,Die Goldene Kamera 2017,8.7,15,"[Steven Gätjen, James Blunt, Leonard Carow, Ge...","[ Riverside Entertainment, Funke Mediengrupp...",[Ladislaus Kiraly],"[awards show, tv special]",[News]
5216,Cedonia Avenue,4.2,11,"[Dwayne Hurt, Durrell Overby, Jaleel Hurt]",[ Amazon Instant Video],[Dwayne Hurt],[game],[Drama]
5217,Autistic Driving School,5.3,10,[Missing],[ Firecracker Films],[Osca Humphreys],[ Plot Keywords submission guide.],[Documentary]
5218,Crap Shoot: The Documentary,3.8,183,"[Kenneth R. Close, James E. Horton, Joey Medin...","[ K.C. Films, Low Fidelity Productions (I)]",[Kenneth R. Close],"[hollywood, science, interview, actress, actor...",[Documentary]
5219,High Strange New Mexico,6.6,7,[Missing],[Missing],[James Lujan],[ Plot Keywords submission guide.],[Documentary]
...,...,...,...,...,...,...,...,...
5290,How I Met Your Mother,8.3,612244,"[Josh Radnor, Jason Segel, Cobie Smulders, Nei...","[ 20Th Century Fox Television, Bays Thomas P...",[Carter Bays],"[group of friends, friendship, best friend, ca...","[Comedy, Romance]"
5291,Sexify,6.4,728,"[Aleksandra Skraba, Maria Sobocinska, Sandra D...",[ Akson Studio],[Aleksandra Skraba],"[partial female nudity, male explicit nudity, ...","[Comedy, Drama]"
5292,Ozark,8.4,196614,"[Jason Bateman, Laura Linney, Sofia Hublitz, S...","[ Zero Gravity Management, Media Rights Capi...",[Bill Dubuque],"[netflix original, missouri, mexico, drug cart...","[Crime, Drama, Thriller]"
5293,365 Days,3.2,57342,"[Anna Maria Sieklucka, Michele Morrone, Bronis...","[ Ekipa, Future Space, Next Film]",[Barbara Bialowas],"[oral sex, sex scene, in love with kidnapper, ...","[Drama, Romance]"


In [ ]:
Netflix2=Netflix1
Netflix2[columnas].apply(lambda x: x.replace("Missing",''))

Num_Actores=[]
for i in Netflix2['Actores']:
  x=int(len(i))
  Num_Actores.append(x)

Netflix2['Num_Actores']=Num_Actores


Num_Etiquetas=[]
for i in Netflix2['Etiquetas']:
  x=int(len(i))
  Num_Etiquetas.append(x)

Netflix2['Num_Etiquetas']=Num_Etiquetas

Num_Género=[]
for i in Netflix2['Género']:
  x=int(len(i))
  Num_Género.append(x)

Netflix2['Num_Género']=Num_Género

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [ ]:
m=[]
for i in range(len(Netflix2['Actores'])):
  if  'missing' in Netflix2['Actores'][i]:
    m.append(i)
media_actores=Netflix2[~Netflix2.index.isin(m)]

n=[]
for i in range(len(Netflix2['Etiquetas'])):
  if  'plotkeywordssubmissionguide.' in Netflix2['Etiquetas'][i]:
    n.append(i)
media_etiqueta=Netflix2[~Netflix2.index.isin(n)]

print("media_actores:",media_actores['Num_Actores'].mean(), "media_género:",Netflix2['Num_Género'].mean(), "media_etiqueta:",media_etiqueta['Num_Etiquetas'].mean() )

   


   
    

media_actores: 12.946435452793834 media_género: 2.518791312559018 media_etiqueta: 39.008717839676805


In [ ]:
def limpiar(x):
  if isinstance(x,list):  #si argumento es una lista
    return [str.lower(i.replace(" ","")) for i in x] #elimina espacios de cada elemento y transforma todo a minusculas
  else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''


In [ ]:
for columna in columnas:
    Netflix1[columna] = Netflix1[columna].apply(limpiar)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:

x=Netflix1.tail(80)

In [ ]:
# for columna in columnas:
#   for i in range(len(Netflix1[columna])):
#     if columna=='Actores':
#        Netflix1[columna][i]= Netflix1[columna][i][:13]
#     if columna=='Genero':
#        Netflix1[columna][i]= Netflix1[columna][i][:]
#     if columna=='Etiquetas':
#        Netflix1[columna][i]= Netflix1[columna][i][:35]
#     else:
#        Netflix1[columna][i]= Netflix1[columna][i][:]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remo

In [ ]:
len(Netflix1["Actores"][1])

38

In [ ]:

Netflix1['Palabras']= Netflix1[columnas].apply(lambda x: ' '.join(x.astype(str)), axis = 1)
Netflix1['Palabras']=Netflix1['Palabras'].apply(lambda x: x.replace('[','').replace(']','').replace("'",'').replace(",",' ').replace("missing",'').replace("plotkeywordssubmissionguide.",''))
Netflix1['Palabras']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0       paulwesley  iansomerhalder  katgraham  candice...
1       lakeithstanfield  mayatanida  williamchristoph...
2       alexanderdreymon  elizabutterworth  ianhart  a...
3       anyataylor-joy  chloepirrie  billcamp  marcind...
4       charliehunnam  kateysagal  markboonejunior  ki...
                              ...                        
5290    joshradnor  jasonsegel  cobiesmulders  neilpat...
5291    aleksandraskraba  mariasobocinska  sandradrzym...
5292    jasonbateman  lauralinney  sofiahublitz  skyla...
5293    annamariasieklucka  michelemorrone  bronislaww...
5294    natsukihanae  zachaguilar  abbytrott  aleksle ...
Name: Palabras, Length: 5295, dtype: object

In [ ]:
conteo = CountVectorizer(stop_words='english') 
matrix = conteo.fit_transform(Netflix1['Palabras'])
sim_coseno= cosine_similarity(matrix, matrix)

In [ ]:
count_vect_df = pd.DataFrame(matrix.todense(), columns=conteo.get_feature_names())
#vector=pd.concat([Netflix["Título"], count_vect_df], axis=1)


In [ ]:
#vector

In [ ]:
Netflix1= Netflix1.reset_index()
Netflix1


,level_0,index,Título,Calificación,Votos,Actores,Productora,Director/Creador,Etiquetas,Género,Palabras
0,0,0,The Vampire Diaries,7.7,287747,"[paulwesley, iansomerhalder, katgraham, candic...","[outerbanksentertainment, alloyentertainment, ...",[julieplec],"[vampire, highschool, teenromance, basedonbook...","[drama, fantasy, horror, mystery, romance, thr...",paulwesley iansomerhalder katgraham candice...
1,1,1,Yasuke,6.3,1157,"[lakeithstanfield, mayatanida, williamchristop...","[mappa, netflix]",[leseanthomas],"[anime, samurai, legend, sword, black, 16thcen...","[animation, action, adventure, drama, fantasy,...",lakeithstanfield mayatanida williamchristoph...
2,2,2,The Last Kingdom,8.4,89263,"[alexanderdreymon, elizabutterworth, ianhart, ...",[carnivalfilm&television],[alexanderdreymon],"[kingdom, danish, anglosaxon, malerearnudity, ...","[action, drama, history]",alexanderdreymon elizabutterworth ianhart a...
3,3,3,The Queen's Gambit,8.6,304534,"[anyataylor-joy, chloepirrie, billcamp, marcin...","[flitcraft, wonderfulfilms, netflix]",[scottfrank],"[addiction, chess, femaleprotagonist, 1950s, o...",[drama],anyataylor-joy chloepirrie billcamp marcind...
4,4,4,Sons of Anarchy,8.5,264918,"[charliehunnam, kateysagal, markboonejunior, k...","[sutterink, linsonentertainment, fox21]",[kurtsutter],"[motorcycle, gunrunning, streetgang, stepfathe...","[crime, drama, thriller]",charliehunnam kateysagal markboonejunior ki...
...,...,...,...,...,...,...,...,...,...,...,...
5290,5290,5290,How I Met Your Mother,8.3,612244,"[joshradnor, jasonsegel, cobiesmulders, neilpa...","[20thcenturyfoxtelevision, baysthomasproductions]",[carterbays],"[groupoffriends, friendship, bestfriend, canad...","[comedy, romance]",joshradnor jasonsegel cobiesmulders neilpat...
5291,5291,5291,Sexify,6.4,728,"[aleksandraskraba, mariasobocinska, sandradrzy...",[aksonstudio],[aleksandraskraba],"[partialfemalenudity, maleexplicitnudity, male...","[comedy, drama]",aleksandraskraba mariasobocinska sandradrzym...
5292,5292,5292,Ozark,8.4,196614,"[jasonbateman, lauralinney, sofiahublitz, skyl...","[zerogravitymanagement, mediarightscapital(mrc)]",[billdubuque],"[netflixoriginal, missouri, mexico, drugcartel...","[crime, drama, thriller]",jasonbateman lauralinney sofiahublitz skyla...
5293,5293,5293,365 Days,3.2,57342,"[annamariasieklucka, michelemorrone, bronislaw...","[ekipa, futurespace, nextfilm]",[barbarabialowas],"[oralsex, sexscene, inlovewithkidnapper, femal...","[drama, romance]",annamariasieklucka michelemorrone bronislaww...


In [ ]:
indices = pd.Series(Netflix1.index, index=Netflix1['Título'])
indices

Título
The Vampire Diaries                  0
Yasuke                               1
The Last Kingdom                     2
The Queen's Gambit                   3
Sons of Anarchy                      4
                                  ... 
How I Met Your Mother             5290
Sexify                            5291
Ozark                             5292
365 Days                          5293
Demon Slayer: Kimetsu no Yaiba    5294
Length: 5295, dtype: int64

In [ ]:
def recomendador():
  argumento=input("Introduce una película o serie: ")
  idx = indices[str(argumento)]
  sim_scores = list(enumerate(sim_coseno[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:50]
  sim = [i[0] for i in sim_scores]
  argumentos= ["Título",'Calificación','Votos']
  return Netflix1[argumentos].iloc[sim].loc[(Netflix1['Calificación']>=(mean(Netflix1['Calificación']))) & (Netflix1['Votos'] >=2000)].head(10)



In [ ]:
argumentos= ["Título",'Calificación','Votos']
Netflix1[argumentos].loc[(Netflix1['Calificación']>=(mean(Netflix1['Calificación']))) & (Netflix1['Votos'] >=1000)].iloc[1:2000].head(10)


,Título,Calificación,Votos
2,The Last Kingdom,8.4,89263
3,The Queen's Gambit,8.6,304534
4,Sons of Anarchy,8.5,264918
5,The 100,7.6,221141
6,"Love, Death & Robots",8.5,102458
8,Community,8.5,224374
10,Suits,8.5,368595
11,Good Girls,7.8,35599
12,The Expanse,8.5,115013
13,Homeland,8.3,318235


In [ ]:
recomendador()

Introduce una película o serie: Rick and Morty


,Título,Calificación,Votos
838,The Midnight Gospel,8.3,12806
26,Family Guy,8.1,308320
511,The Boondocks,8.4,27146
8,Community,8.5,224374
58,BoJack Horseman,8.7,117763
386,Disenchantment,7.2,48979
981,The Windsors,7.1,2268
633,The Amazing World of Gumball,8.2,22906
214,Big Mouth,8.0,59186
878,Chappelle's Show,8.8,56005


In [ ]:
recomendador()

Introduce una película o serie: Rick and Morty


,Título,Calificación,Votos
838,The Midnight Gospel,8.3,12806
511,The Boondocks,8.4,27146
8,Community,8.5,224374
58,BoJack Horseman,8.7,117763
214,Big Mouth,8.0,59186
5274,The Mitchells vs the Machines,8.0,10721
981,The Windsors,7.1,2268
713,Close Enough,7.8,4751
633,The Amazing World of Gumball,8.2,22906
386,Disenchantment,7.2,48979


In [ ]:
	print(mean(Netflix1["Calificación"]), mean(Netflix1["Votos"]))

6.603456090651558 21748.460245514638


In [ ]:
x=[]
for i in Netflix1["Título"]:
  x.append(i)

x=sorted(x)
for i in x:
  print(i)

Streaming output truncated to the last 5000 lines.
Amo
Amy Schumer: Growing
Amy Schumer: The Leather Special
An Act of War
An Affair
An Affair to Die For
An Anthology of Fellings
An Easy Girl
An Interview with God
An Unremarkable Christmas
An Upper Egyptian
Anatomy of a Love Seen
Ancient Extraterrestrials: Aliens and UFOs Before the Dawn of Time
And Breathe Normally
And Tomorrow the Entire World
And the Oscar Goes To...
Andes Mágicos
André & His Olive Tree
Anelka: Misunderstood
Aneun Waipeu
Ang probinsyano
Angel 'n' Devil
Angela's Christmas
Angela's Christmas Wish
Angels of Death
Angry Birds Blues
Angry Birds Stella
Angry Birds Toons
Angry Indian Goddesses
Angry Mom
Anima
Animal Crackers
Animal Kingdom
Animal World
Animals on the Loose: A You vs. Wild Movie
Anitta: Made in Honório
Anjaan: Special Crimes Unit
Anjelah Johnson: Not Fancy
Anne with an E
Annihilation
Anon
Another
Another Life
Another Miss Oh
Answer for Heaven
Anthony Jeselnik: Fire in the Maternity Ward
Anthony Jeselnik: Th

That'